<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Weather Classification Assignment**


Estimated time needed: **60** minutes


In this notebook, we will practice using all the classification algorithms and metrics that we learned in this course. Using weather data we will try to predict if there is going to rain the next day.


## Objectives


After completing this lab you will be able to:


*   Data
    *   Describe and Define the Dataset
    *   Load a CSV Dataset using Pandas
    *   Preprocess the Data using Pandas
    *   Deal with NULL Values in your Dataset
    *   Perform One Hot Encoding on Categorical Variables
    *   Split your Data into a Training and Testing Set
    *   Standardize your Data using StandardScaler or MinMax
*   Classification
    *   Use GridSearchCV to Find the Best Parameters for a Classification Algorithm
    *   Perform Classification using Logistic Regression
    *   Perform Classification using K-Nearest Neighbors
    *   Perform Classification using Support Vector Machine
    *   Perform Classification using Decision Trees
*   Use Evaluation Metrics Accuracy Score, Jaccard Index, F1-Score, and Log Loss on Each Algorithm and Report the Results


***


## Setup


First, we will download the data that we will use in this lab which is stored in a CSV format.


In [35]:
from js import fetch
import io

URL = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/ML0101ENv3/project_EdX/weatherAUS.csv"
resp = await fetch(URL)
text = io.BytesIO((await resp.arrayBuffer()).to_py())

For this lab, we are going to be using Python and several Python libraries.

If you are running this Jupyter Notebook locally, you need to install the following libraries by uncommenting the code bellow. Otherwise, leave the code bellow commented out and run the rest of this notebook.


In [36]:
#pip install pandas
#pip install sklearn
#pip install matplotlib
#pip install numpy

In [37]:
# allows us to interact with the data using a dataframe
import pandas as pd
# allows us to interact with the data and perform calculations using ndarrays
import numpy as np
# various classification algorithms and metrics from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
# matplotlib allows us to create graphs
import matplotlib.pyplot as plt

Since sklearn calculates jaccard index differently than what was taught in the course we will define our own function for jaccard index


In [38]:
# works like sklearn classificaton metrics given list or ndarray of predictions and values returns the jaccar index
def jaccard_index(predictions, true):
    if (len(predictions) == len(true)):
        intersect = 0;
        for x,y in zip(predictions, true):
            if (x == y):
                intersect += 1
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

## Data


### About the Data


The original source of the data is Austrailian Government's Bureau of Meteorology and the latest data can be gathered from [http://www.bom.gov.au/climate/dwo/](http://www.bom.gov.au/climate/dwo/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01).

The dataset we will use has extra columns like RainToday and our target RainTomorrow which was gathered from Rattle at [https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData](https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


This dataset is observations of weather metrics for each day from 2008 to 2017. The **weatherAUS.csv** dataset includes the following fields:

| Field         | Description                                           | Unit            | Type   |
| ------------- | ----------------------------------------------------- | --------------- | ------ |
| Date          | Date of the Observation in YYYY-MM-DD                 | Date            | object |
| Location      | Location of the Observation                           | Location        | object |
| MinTemp       | Minimum temperature                                   | Celsius         | float  |
| MaxTemp       | Maximum temperature                                   | Celsius         | float  |
| Rainfall      | Amount of rainfall                                    | Millimeters     | float  |
| Evaporation   | Amount of evaporation                                 | Millimeters     | float  |
| Sunshine      | Amount of bright sunshine                             | hours           | float  |
| WindGustDir   | Direction of the strongest gust                       | Compass Points  | object |
| WindGustSpeed | Speed of the strongest gust                           | Kilometers/Hour | object |
| WindDir9am    | Wind direction averaged 10 minutes prior to 9am       | Compass Points  | object |
| WindDir3pm    | Wind direction averaged 10 minutes prior to 3pm       | Compass Points  | object |
| WindSpeed9am  | Wind speed averaged 10 minutes prior to 9am           | Kilometers/Hour | float  |
| WindSpeed3pm  | Wind speed averaged 10 minutes prior to 3pm           | Kilometers/Hour | float  |
| Humidity9am   | Humidity at 9am                                       | Percent         | float  |
| Humidity3pm   | Humidity at 3pm                                       | Percent         | float  |
| Pressure9am   | Atmospheric pressure reduced to mean sea level at 9am | Hectopascal     | float  |
| Pressure3pm   | Atmospheric pressure reduced to mean sea level at 3pm | Hectopascal     | float  |
| Cloud9am      | Fraction of the sky obscured by cloud at 9am          | Eights          | float  |
| Cloud3pm      | Fraction of the sky obscured by cloud at 3pm          | Eights          | float  |
| Temp9am       | Temperature at 9am                                    | Celsius         | float  |
| Temp3pm       | Temperature at 3pm                                    | Celsius         | float  |
| RainToday     | If there was rain today                               | Yes/No          | object |
| RISK_MM       | Amount of rain tomorrow                               | Millimeters     | float  |
| RainTomorrow  | If there is rain tomorrow                             | Yes/No          | float  |

Column definitions were gathered from [http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml](http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


### Load the Dataset


Lets use the **head()** function to see our data


In [39]:
df = pd.read_csv(text)

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


### Preprocessing


We want to focus specifically on Sydney so that we can train our algorithm quickly. You can select other locations or multiple locations if you would like to experiment.


In [40]:
df = df[df['Location'] == 'Sydney']

Next, we drop all the columns in the table that we won't need.

We drop Location because it is constant for each row and we drop RIS_MM because this tells us the amount of rain tomorrow so we can not train on it as it reveals the target and we are doing classification, not regression.


In [41]:
df_sydney = df[df['Location'] == 'Sydney']

df_sydney.drop(columns=['Location', 'RISK_MM'], axis=1, inplace=True)

print(df_sydney.shape)

df_sydney.head()

(3337, 22)


,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
29497,2008-02-01,19.5,22.4,15.6,6.2,0.0,NaN,NaN,S,SSW,...,92.0,84.0,1017.6,1017.4,8.0,8.0,20.7,20.9,Yes,Yes
29498,2008-02-02,19.5,25.6,6.0,3.4,2.7,NaN,NaN,W,E,...,83.0,73.0,1017.9,1016.4,7.0,7.0,22.4,24.8,Yes,Yes
29499,2008-02-03,21.6,24.5,6.6,2.4,0.1,NaN,NaN,ESE,ESE,...,88.0,86.0,1016.7,1015.6,7.0,8.0,23.5,23.0,Yes,Yes
29500,2008-02-04,20.2,22.8,18.8,2.2,0.0,NaN,NaN,NNE,E,...,83.0,90.0,1014.2,1011.8,8.0,8.0,21.4,20.9,Yes,Yes
29501,2008-02-05,19.7,25.7,77.4,NaN,0.0,NaN,NaN,NNE,W,...,88.0,74.0,1008.3,1004.8,8.0,8.0,22.5,25.5,Yes,Yes


As you can see above we have NaN occur a couple of times in our dataset. We can either drop the data or replace the data.


Below we can see how many NaN values we have for each row. WindGustDir, WindGustSpeed, Cloud9am, and Cloud3pm have large values of missing data. In this case for \~33% of the data, we are missing a value for WindGusDir and WindGustSpeed. This is not enough to remove the entire column but we will perform some preprocessing.


In [42]:
df_sydney.isna().sum()

Date                0
MinTemp             3
MaxTemp             2
Rainfall            6
Evaporation        51
Sunshine           16
WindGustDir      1036
WindGustSpeed    1036
WindDir9am         56
WindDir3pm         33
WindSpeed9am       26
WindSpeed3pm       25
Humidity9am        14
Humidity3pm        13
Pressure9am        20
Pressure3pm        19
Cloud9am          566
Cloud3pm          561
Temp9am             4
Temp3pm             4
RainToday           6
RainTomorrow        0
dtype: int64

### Dealing With Nulls


Please uncomment the method that you would like to use

1.  Drop all rows that contain NaN
2.  Replace NaN in object type columns like WindGustDir with most frequent value in the column and replace NaN in float type columns like WindGustSpeed, Cloud9am, and Cloud3pm with the mean. Then we drop the remaining rows with NaN in them.

Please note that if you choose to replace the NaN values the classification algorithms will take a little longer to compute


1.  Drop


In [43]:
df_sydney_filled = df_sydney.dropna()

2.  Replace


In [44]:
df_sydney_filled = df_sydney.copy()

most_frequent_WindGustDir = df_sydney_filled['WindGustDir'].value_counts().idxmax()
df_sydney_filled["WindGustDir"].replace(np.nan, most_frequent_WindGustDir, inplace=True)

mean_WindGustSpeed = df_sydney_filled["WindGustSpeed"].astype("float").mean(axis=0)
df_sydney_filled["WindGustSpeed"].replace(np.nan, mean_WindGustSpeed, inplace=True)

mean_Cloud9am = df_sydney_filled["Cloud9am"].astype("float").mean(axis=0)
df_sydney_filled["Cloud9am"].replace(np.nan, mean_Cloud9am, inplace=True)

mean_Cloud3pm = df_sydney_filled["Cloud3pm"].astype("float").mean(axis=0)
df_sydney_filled["Cloud3pm"].replace(np.nan, mean_Cloud3pm, inplace=True)

df_sydney_filled.dropna(inplace=True)

In [45]:
print(df_sydney_filled.shape)
df_sydney_filled.isna().sum()

(3155, 22)


Date             0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

As you can see we have completely removed all NaN values using different methods which allow you to either remove rows with NaN in them improving the pureness of our dataset or filling in NaN values allowing us to preserve rows. When deciding on the method to use there are many benefits and drawbacks we must consider like whether or not we will have enough data after dropping NaN rows or if filling in Nan by frequency or mean will introduce some sort of bias to our data.


In [46]:
df_sydney_filled.loc[:,'Date'] = df['Date'].str.replace('-', '')

Finally, we remove the - between the values of the Date column so they can be converted to floats


### One Hot Encoding


Finally we need to perform one hot encoding to convert categorical variables to binary variables


In [47]:
df_sydney_processed = pd.get_dummies(data=df_sydney_filled, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

Next, we replace the values of the RainTomorrow column changing it from a categorical column to a binary column. We do not use the **get_dummies** method because we would end up with two columns for RainTomorrow and we do not want that because it is our target.


In [48]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

### Training Data and Testing Data


First, we turn all columns into a float type. We don't need to do this because the **StandardScalar()** method will convert object types to float but it will give us a warning message.


In [49]:
df_sydney_processed = df_sydney_processed.astype(float)

Now we split our dataset into a features dataset and target dataset. We drop our target to create our features dataset and only keep RainTomorrow to create our target dataset


In [50]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

Now we will standardize the data. We can do this in multiple ways like using the **StandardScalar()** method which will scale the values to unit variance or the **MinMaxScalar()** which will scale each value to the min and max of each column.


### Data Standardization


Before we standardize our data we must split it into training and testing sets. We do this before standarsizing so that we don't give any hints to out model by standardizing all the data together.


In [51]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=.2, random_state=1)

Please uncomment the method you would like to choose


1.


In [52]:
norm = preprocessing.StandardScaler()

2.


In [53]:
norm = preprocessing.MinMaxScaler()

In [54]:
x_train = norm.fit_transform(x_train)

x_test = norm.transform(x_test)

As we discussed before you can see how we fit and the scaler to the training data and also transformed it. Then we used the fitted scaler to transform the test data.


## Classification


### Instructions


Below is where we are going to use the classification algorithms to create a model based on our training data and finally evaluate our testing data using evaluation metrics learned in the course

We will some of the algorithms taught in the course, specifically

1.  Logistic Regression
2.  KNN
3.  SVM
4.  Decision Trees

We will evaluate our models using

1.  Accuracy Score
2.  Jaccard Index
3.  F1-Score
4.  Log Loss

Note: Jaccard Index is calculated differently in Sci Kit Learn so I have defined a function at the top of the notebook for you to use, its input style is the same as Sci Kit Learn

As we know these algorithms have many parameters and to find the best ones we will use GridSearchCV

I will demonstrate how to do this using a mock classification algorithm

1.  Create a python dictionary with the key being the name of the parameters and the value being a list of possible values
2.  Create an object of the classification algorithm
3.  Create a GridSearchCV object and place your classification object and parameters dictionary as parameters, also define your GridSearchCV cv parameter (Use cv = 4)
4.  Use the fit method of the GridSearchCV algorithm to train our model using x_train and y_train that we create before
5.  Store the best model in a variable provided
6.  Predict the target variable using the x_test data we created above
7.  Calculate and store the values for each metric in the provided variables using the predictions and y_test data

You will need to research the parameters you need to use as there are many options but this is simple. GridSearchCV will determine the best model.

Finally using your models generate the report at the bottom


### Mock


If you need some more help with grid search here are a couple of resources

1.  [https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)
2.  [https://scikit-learn.org/stable/modules/grid_search.html](https://scikit-learn.org/stable/modules/grid_search.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


### Question 1: Logistic Regression


For Logistic Regression please use the parameters C = \[.001, .01, .1, 1, 10, 100] and solver. Use the link provided to select the values for the solver parameter. [https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


When creating the LogisticRegression object please make **max_iter = 10000**. This will allow us enough iteration so the model parameters can converge


In [55]:
BestLR = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-55-fabeda319123>, line 1)

In [56]:
print(BestLR)

<class 'NameError'>: name 'BestLR' is not defined

In [57]:
LR_Accuracy_Score = 
LR_JaccardIndex = 
LR_F1_Score = 
LR_Log_Loss = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-57-54a47c5fa727>, line 1)

### Question 2: KNN


For KNN please use the parameters n_neighbors = \[1,2,3,4,5,6,7,8,9,10], algorithm, and p. Use the link provided to select the values for algorithm and p. [https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


In [58]:
BestKNN =

<class 'SyntaxError'>: invalid syntax (<ipython-input-58-ce828ee08b62>, line 1)

In [59]:
print(BestKNN)

<class 'NameError'>: name 'BestKNN' is not defined

In [60]:
KNN_Accuracy_Score = 
KNN_JaccardIndex = 
KNN_F1_Score = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-60-80e78d7d019d>, line 1)

### Question 3: SVM


For SVM please use the parameters C = \[.001, .01, .1, 1, 10, 100] and kernel. Use the link provided to select the values for kernel. [https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


In [61]:
BestSVM =

<class 'SyntaxError'>: invalid syntax (<ipython-input-61-7b2ce638c8c7>, line 1)

In [62]:
print(BestSVM)

<class 'NameError'>: name 'BestSVM' is not defined

In [63]:
SVM_Accuracy_Score = 
SVM_JaccardIndex = 
SVM_F1_Score = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-63-7f39845ec73c>, line 1)

### Question 4: Decision Tree


For Decision Tree please use the parameters criterion. Use the link provided to select the values for criterion. [https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


In [64]:
BestTree =

<class 'SyntaxError'>: invalid syntax (<ipython-input-64-40a6b14f4723>, line 1)

In [65]:
print(BestTree)

<class 'NameError'>: name 'BestTree' is not defined

In [66]:
Tree_Accuracy_Score = 
Tree_JaccardIndex = 
Tree_F1_Score = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-66-23f6fc7f70df>, line 1)

## Report


In [67]:
Report = pd.DataFrame({'Algorithm' : ['KNN', 'Decision Tree', 'SVM', 'LogisticRegression']})

Report['Accuracy'] = [LR_Accuracy_Score, KNN_Accuracy_Score, SVM_Accuracy_Score, Tree_Accuracy_Score]
Report['Jaccard'] = [LR_JaccardIndex, KNN_JaccardIndex, SVM_JaccardIndex, Tree_JaccardIndex]
Report['F1-Score'] = [LR_F1_Score, KNN_F1_Score, SVM_F1_Score, Tree_F1_Score]
Report['LogLoss'] = ['N/A', 'N/A', 'N/A', LR_Log_Loss]

Report

<class 'NameError'>: name 'LR_Accuracy_Score' is not defined

## Authors


[Azim Hirjani](https://www.linkedin.com/in/azim-hirjani-691a07179/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description         |
| ----------------- | ------- | ---------- | -------------------------- |
| 2020-09-14        | 0.2     | Azim       | Update Lab to Use Template |
| 2020-04-17        | 0.1     | Azim       | Created Lab                |


Copyright © 2020 IBM Corporation. All rights reserved.
